In [1]:
def LHT(GRAN, op, l, r):
    if op in Locks[GRAN][l]:
        return service_time[op] + lock_cost[modes[GRAN][MODE][op][l]][place[GRAN][PLACE][l]][r]
    else:
        return 0

    
def op_serialization(GRAN,r):
    res = 0
    for l in Locks[GRAN]:
        for op in Locks[GRAN][l]:
            if modes[GRAN][MODE][op][l] == 'X':
                for r_1 in Reps:
                    if r_1 != r:
                        res += LHT(GRAN, op, l, r_1) * freq[WL][op][r_1]
    return res


def rep_serialization(GRAN,r):
    res = 0
#     for op in Locks[l]:
#         res += LHT(op, l, r) * freq[WL][op][r]
    for l in Locks[GRAN]:
        for op in freq[WL]:
            for l in Locks[GRAN]:
                if op in Locks[GRAN][l]:
                    res += LHT(GRAN, op, l, r) * freq[WL][op][r]
    return res


def op_parallelism(GRAN,r):
    res = 0
    s = 0
    for l in Locks[GRAN]:
        for r_1 in Reps:
            if r != r_1:
                for op in Locks[GRAN][l]:
                    if modes[GRAN][MODE][op][l] == 'S':
                        s += LHT(GRAN, op, l, r_1) * freq[WL][op][r_1]
            res = max(res, s)
#             res += s
#     print('op', res)
    return res
#     return res/(len(Reps)-1)


def nonparallelism(GRAN):
    res = 0
    for r in Reps:
        np = abs(op_serialization(GRAN, r) + rep_serialization(GRAN, r) - op_parallelism(GRAN, r))
        res = max(res, np)
    return res


# def coordination_parallelism():
#     res = 0
#     for l in Locks:
#         np = nonparallelism(l)
#         res = max(res, np)
#     return res

def amdahl(GRAN):
    res = 0
    for r in Reps:
        np = op_serialization(GRAN, r) + rep_serialization(GRAN, r) + op_parallelism(GRAN, r)
        res += np
#     res = max(res, np)
    return res/len(Reps)



In [2]:
Reps = ['houston', 'paris', 'singapore']
Ops = ['insert', 'read', 'update']
service_time = {'insert':0, 'read':0, 'update':0}
lock_cost = {'X': {'houston':{'houston':3.2954730850000002, 'paris':223.78096148900002, 'singapore':503.93539986400003}, 
                     'paris':{'houston':223.99058557599997, 'paris':3.2398566490000005, 'singapore':303.88539813499995}, 
                     'singapore':{'houston':503.86683250899995, 'paris':303.965657338, 'singapore':3.108215382}},
             'S': {'houston':{'houston':3.0035555910000005, 'paris':223.53921476500003, 'singapore':503.69265980899996}, 
                     'paris':{'houston':223.67417935400002, 'paris':3.0147095129999997, 'singapore':303.628741349}, 
                     'singapore':{'houston':503.62028737099996, 'paris':303.671423894, 'singapore':2.9168805460000002}}
            }
freq = {'workloadeqeq':{'insert':{'houston':111, 'paris':111, 'singapore':111}, 
                        'read':{'houston':111, 'paris':111, 'singapore':111},
                       'update':{'houston':111, 'paris':111, 'singapore':111}},
       'workloadeqhot':{'insert':{'houston':0, 'paris':333, 'singapore':0}, 
                        'read':{'houston':0, 'paris':333, 'singapore':0},
                       'update':{'houston':0, 'paris':333, 'singapore':0}},
        'workloadeqclust':{'insert':{'houston':167, 'paris':167, 'singapore':0}, 
                           'read':{'houston':167, 'paris':167, 'singapore':0},
                          'update':{'houston':167, 'paris':167, 'singapore':0}},
        'workloadabceq':{'insert':{'houston':303, 'paris':303, 'singapore':303}, 
                     'read':{'houston':25, 'paris':25, 'singapore':25},
                    'update':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadabchot':{'insert':{'houston':0, 'paris':909, 'singapore':0}, 
                      'read':{'houston':0, 'paris':75, 'singapore':0},
                     'update':{'houston':0, 'paris':15, 'singapore':0}},
        'workloadabcclust':{'insert':{'houston':455, 'paris':455, 'singapore':0}, 
                    'read':{'houston':37, 'paris':37, 'singapore':0},
                   'update':{'houston':8, 'paris':8, 'singapore':0}},
        'workloadbaceq':{'insert':{'houston':25, 'paris':25, 'singapore':25}, 
                     'read':{'houston':303, 'paris':303, 'singapore':303},
                    'update':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadbachot':{'insert':{'houston':0, 'paris':75, 'singapore':0}, 
                      'read':{'houston':0, 'paris':909, 'singapore':0},
                     'update':{'houston':0, 'paris':15, 'singapore':0}},
        'workloadbacclust':{'insert':{'houston':37, 'paris':37, 'singapore':0}, 
                    'read':{'houston':455, 'paris':455, 'singapore':0},
                   'update':{'houston':8, 'paris':8, 'singapore':0}}
       }


Locks = {1:{'a-b':{'insert', 'read'}, 'b-c':{'read', 'update'}},
         2:{'a-b-c':{'insert', 'read', 'update'}}}

modes = {1:{1:{'insert':{'a-b':'X'}, 'read':{'a-b':'X', 'b-c':'X'}, 'update':{'b-c':'X'}}, 
             2:{'insert':{'a-b':'X'}, 'read':{'a-b':'X', 'b-c':'X'}, 'update':{'b-c':'S'}},
             3:{'insert':{'a-b':'X'}, 'read':{'a-b':'X', 'b-c':'S'}, 'update':{'b-c':'X'}},
             4:{'insert':{'a-b':'X'}, 'read':{'a-b':'S', 'b-c':'X'}, 'update':{'b-c':'X'}},
             5:{'insert':{'a-b':'X'}, 'read':{'a-b':'S', 'b-c':'X'}, 'update':{'b-c':'S'}},
             6:{'insert':{'a-b':'X'}, 'read':{'a-b':'S', 'b-c':'S'}, 'update':{'b-c':'X'}},
             7:{'insert':{'a-b':'S'}, 'read':{'a-b':'X', 'b-c':'X'}, 'update':{'b-c':'X'}},
             8:{'insert':{'a-b':'S'}, 'read':{'a-b':'X', 'b-c':'X'}, 'update':{'b-c':'S'}},
             9:{'insert':{'a-b':'S'}, 'read':{'a-b':'X', 'b-c':'S'}, 'update':{'b-c':'X'}}},
         2:{1:{'insert':{'a-b-c':'X'}, 'read':{'a-b-c':'X'}, 'update':{'a-b-c':'X'}},
         2:{'insert':{'a-b-c':'X'}, 'read':{'a-b-c':'S'}, 'update':{'a-b-c':'X'}},
         3:{'insert':{'a-b-c':'S'}, 'read':{'a-b-c':'X'}, 'update':{'a-b-c':'S'}}}
        }
place = {1:{1:{'a-b':'houston', 'b-c':'houston'},
             2:{'a-b':'paris', 'b-c':'houston'},
             3:{'a-b':'singapore', 'b-c':'houston'},
             4:{'a-b':'houston', 'b-c':'paris'},
             5:{'a-b':'paris', 'b-c':'paris'},
             6:{'a-b':'singapore', 'b-c':'paris'},
             7:{'a-b':'houston', 'b-c':'singapore'},
             8:{'a-b':'paris', 'b-c':'singapore'},
             9:{'a-b':'singapore', 'b-c':'singapore'}},
         2:{1:{'a-b-c':'houston'},
           2:{'a-b-c':'paris'},
           3:{'a-b-c':'singapore'}}
        }


In [3]:
predictions = {}
for WL in freq:
    predictions[WL] = {}
    for GRAN in Locks:
        for MODE in modes[GRAN]:
            for PLACE in place[GRAN]:
                predictions[WL][str(GRAN) + '-' + str(MODE) + '-' + str(PLACE)] = nonparallelism(GRAN)
#             print(str(MODE) + '-' + str(i+1), '\t', coordination_parallelism())
#             print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', coordination_parallelism())
#             print(coordination_parallelism())
print(predictions)
       


{'workloadeqeq': {'1-1-1': 548316.572030088, '1-1-2': 459528.56096093403, '1-1-3': 459473.213185668, '1-1-4': 459528.5609609339, '1-1-5': 370740.5498917799, '1-1-6': 459520.8999356279, '1-1-7': 459473.213185668, '1-1-8': 459520.89993562794, '1-1-9': 583774.5467556719, '1-2-1': 497910.951990648, '1-2-2': 409122.9409214939, '1-2-3': 404471.526351246, '1-2-4': 409086.53740323, '1-2-5': 320298.52633407596, '1-2-6': 391322.3314276379, '1-2-7': 386732.260962546, '1-2-8': 391349.665250232, '1-2-9': 515603.3120702759, '1-3-1': 497910.951990648, '1-3-2': 409122.9409214939, '1-3-3': 404471.526351246, '1-3-4': 409086.53740323, '1-3-5': 320298.52633407596, '1-3-6': 391322.3314276379, '1-3-7': 386732.260962546, '1-3-8': 391349.665250232, '1-3-9': 515603.3120702759, '1-4-1': 497910.951990648, '1-4-2': 409086.5374032301, '1-4-3': 386732.260962546, '1-4-4': 409122.9409214939, '1-4-5': 320298.52633407596, '1-4-6': 391349.6652502319, '1-4-7': 404471.526351246, '1-4-8': 391322.33142763795, '1-4-9': 51560

In [4]:
amd = {}
for WL in freq:
    amd[WL] = {}
    for GRAN in Locks:
        for MODE in modes[GRAN]:
            for PLACE in place[GRAN]:
                amd[WL][str(GRAN) + '-' + str(MODE) + '-' + str(PLACE)] = amdahl(GRAN)
print(amd)

{'workloadeqeq': {'1-1-1': 432759.00598729606, '1-1-2': 373589.791740208, '1-1-3': 456417.951741432, '1-1-4': 373589.79174020793, '1-1-5': 314420.5774931199, '1-1-6': 397248.73749434407, '1-1-7': 456417.951741432, '1-1-8': 397248.7374943439, '1-1-9': 480076.89749556786, '1-2-1': 432644.09815489204, '1-2-2': 373474.883907804, '1-2-3': 456303.043909028, '1-2-4': 373471.6566388959, '1-2-5': 314302.44239180797, '1-2-6': 397130.60239303205, '1-2-7': 456309.598955568, '1-2-8': 397140.38470848, '1-2-9': 479968.54470970394, '1-3-1': 432644.098154892, '1-3-2': 373474.883907804, '1-3-3': 456303.04390902794, '1-3-4': 373471.65663889603, '1-3-5': 314302.44239180797, '1-3-6': 397130.60239303205, '1-3-7': 456309.598955568, '1-3-8': 397140.38470848, '1-3-9': 479968.54470970394, '1-4-1': 432644.09815489204, '1-4-2': 373471.65663889603, '1-4-3': 456309.598955568, '1-4-4': 373474.883907804, '1-4-5': 314302.44239180797, '1-4-6': 397140.38470848, '1-4-7': 456303.043909028, '1-4-8': 397130.602393032, '1-4-